## Deriving the Covariance relationships of Depolarizing Channel

Given the Hamiltonain only depends on $R_xyz$


and 
$$ U = cos(\theta s /2) I - i sin(\theta s /2) R $$

Where $$ R_{xyz} = sin(\psi) cos(\phi) X + sin(\psi) sin(\phi) Y + cos(\psi) Z $$

Goal to Compute X, Y, Z components for each X, Y, Z, I gate
H ommited since $$ H= \dfrac{1}{\sqrt 2} (X+Z)$$

| Gate  | $\theta$ | $\psi$          | $\phi$          |
| ----- | -------- | --------------- | --------------- |
| **I** | $2\pi$   | –               | –               |
| **X** | –        | $\frac{\pi}{2}$ | $0$             |
| **Y** | –        | $\frac{\pi}{2}$ | $\frac{\pi}{2}$ |
| **Z** | –        | $0$             | $\frac{\pi}{2}$ |


In [1]:
import sympy as sp
import numpy as np
from numpy import exp, sqrt, array
from numpy.random import multivariate_normal

In [2]:

# === Symbols ===
theta, s = sp.symbols('theta s', real=True)
alpha = s * theta / 2

i = sp.I

# === Pauli Matrices ===
I2 = sp.eye(2)
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -i], [i, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])
pauli_labels = {'I': I2, 'X': X, 'Y': Y, 'Z': Z}



def compute_R(psi, phi):
    # === Rotation Axes ===
    R = (
        sp.sin(psi) * sp.cos(phi) * X +
        sp.sin(psi) * sp.sin(phi) * Y +
        sp.cos(psi) * Z
    )
    return R

# === IBM-style unitary ===
def compute_U(alpha, R):
    U = sp.cos(alpha) * I2 - sp.I * sp.sin(alpha) * R
    U_dag = U.H
    return U, U_dag

# === Core function: compute f_k^σ(s) components ===
def compute_pauli_components(sigma_k, U, U_dag):
    Lk = sp.simplify(U_dag * sigma_k * U)
    components = {}
    for label, P in pauli_labels.items():
        f_σ = sp.simplify(sp.trigsimp((Lk * P).trace() / 2))
        components[label] = f_σ
    return Lk, components

# === Integration helper for variances and covariances ===
def compute_var_and_cov(f_dict):
    keys = list(f_dict.keys())
    results = {}
    for k in keys:
        results[f'Var_{k}'] = sp.simplify(sp.integrate(f_dict[k]**2, (s, 0, 1)))
    for i in range(len(keys)):
        for j in range(i+1, len(keys)):
            k1, k2 = keys[i], keys[j]
            cov = sp.simplify(sp.integrate(f_dict[k1] * f_dict[k2], (s, 0, 1)))
            results[f'Cov_{k1}_{k2}'] = cov
    return results



In [3]:
def analyze_pauli_rotation(pauli_target, psi, phi, alpha=alpha):
    # Identify label dynamically
    inv_pauli_labels = {id(mat): label for label, mat in pauli_labels.items()}
    pauli_target_label = inv_pauli_labels.get(id(pauli_target), "Unknown")

    # Compute L_k and its Pauli decomposition
    R = compute_R(psi, phi)
    U, U_dag = compute_U(alpha, R)
    Lk, f_components = compute_pauli_components(pauli_target,U, U_dag)
    variances = compute_var_and_cov(f_components)

    # Print results
    print(f"\n=== Analysis for Pauli Target: {pauli_target_label} ===")
    print(f"\n--- Matrix L_k = U† {pauli_target_label} U ---")
    print(Lk)  # cleaner symbolic form


    print(f"\n--- Pauli Basis Components f_k^({pauli_target_label})(s) ---")
    for label, expr in f_components.items():
        print(f"f_{pauli_target_label}^{label}(s) =", expr)

    print(f"\n--- Variances and Covariances for L_k = {pauli_target_label} ---")
    for name, val in variances.items():
        print(f"{name} =", val)
        
    return Lk, f_components, variances


Z -component

In [4]:
## Z - Target Depolarizing Component ##
def z_depolarizing_component(psi,phi,alpha):
    
    Lk_Z, f_components_Z, variances_Z = analyze_pauli_rotation(Z,psi,phi,alpha)  # func call

    # Extract relevant entries for cov_x matrix
    VarZ_X = variances_Z['Var_X']
    VarZ_Y = variances_Z['Var_Y']
    VarZ_Z = variances_Z['Var_Z']
    CovZ_XY = variances_Z.get('Cov_X_Y', 0)
    CovZ_XZ = variances_Z.get('Cov_X_Z', 0)
    CovZ_YZ = variances_Z.get('Cov_Y_Z', 0)


    cov_Z_symbolic = sp.Matrix([[VarZ_X, CovZ_XY, CovZ_XZ],
                                [CovZ_XY, VarZ_Y, CovZ_YZ],
                                [CovZ_XZ, CovZ_YZ, VarZ_Z]])

    cov_Z_symbolic = sp.simplify(cov_Z_symbolic)

    return cov_Z_symbolic

X-component

In [5]:
## X - Target Depolarizing Component ##
# Compute and extract symbolic variances/covariances for Pauli X
def x_depolarizing_component(psi,phi,alpha):
    Lk_X, f_components_X, variances_X = analyze_pauli_rotation(X,psi,phi,alpha)  # func call

    # Extract relevant entries for cov_x matrix
    VarX_X = variances_X['Var_X']
    VarX_Y = variances_X['Var_Y']
    VarX_Z = variances_X['Var_Z']
    CovX_XY = variances_X.get('Cov_X_Y', 0)
    CovX_XZ = variances_X.get('Cov_X_Z', 0)
    CovX_YZ = variances_X.get('Cov_Y_Z', 0)

    # Construct symbolic covariance matrix
    cov_x_symbolic = sp.Matrix([
        [VarX_X,    CovX_XY,   CovX_XZ],
        [CovX_XY,   VarX_Y,    CovX_YZ],
        [CovX_XZ,   CovX_YZ,   VarX_Z]
    ])

    cov_x_symbolic = sp.simplify(cov_x_symbolic)

    return cov_x_symbolic

Y-component

In [6]:
## Y - Target Depolarizing Component ##
def y_depolarizing_component(psi,phi,alpha):
    Lk_Y, f_components_Y, variances_Y = analyze_pauli_rotation(Y, psi,phi,alpha)

    VarY_X = variances_Y['Var_X']
    VarY_Y = variances_Y['Var_Y']
    VarY_Z = variances_Y['Var_Z']
    CovY_XY = variances_Y.get('Cov_X_Y', 0)
    CovY_XZ = variances_Y.get('Cov_X_Z', 0)
    CovY_YZ = variances_Y.get('Cov_Y_Z', 0)

    cov_Y_symbolic = sp.Matrix([
        [VarY_X, CovY_XY, CovY_XZ],
        [CovY_XY, VarY_Y, CovY_YZ],
        [CovY_XZ, CovY_YZ, VarY_Z]
    ])

    cov_Y_symbolic = sp.simplify(cov_Y_symbolic)

    return cov_Y_symbolic

# X - Gate

In [7]:
## Change for Each Gate implemented ##
psi = sp.pi / 2
phi = 0

x_depolarizing_component(psi,phi,alpha)



=== Analysis for Pauli Target: X ===

--- Matrix L_k = U† X U ---
Matrix([[0, 1], [1, 0]])

--- Pauli Basis Components f_k^(X)(s) ---
f_X^I(s) = 0
f_X^X(s) = 1
f_X^Y(s) = 0
f_X^Z(s) = 0

--- Variances and Covariances for L_k = X ---
Var_I = 0
Var_X = 1
Var_Y = 0
Var_Z = 0
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = 0
Cov_Y_Z = 0


Matrix([
[1, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [8]:
z_depolarizing_component(psi,phi,alpha)



=== Analysis for Pauli Target: Z ===

--- Matrix L_k = U† Z U ---
Matrix([[cos(s*theta), -I*sin(s*theta)], [I*sin(s*theta), -cos(s*theta)]])

--- Pauli Basis Components f_k^(Z)(s) ---
f_Z^I(s) = 0
f_Z^X(s) = 0
f_Z^Y(s) = sin(s*theta)
f_Z^Z(s) = cos(s*theta)

--- Variances and Covariances for L_k = Z ---
Var_I = 0
Var_X = 0
Var_Y = Piecewise(((theta - sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Var_Z = Piecewise(((theta + sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (1, True))
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = 0
Cov_Y_Z = Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))


Matrix([
[0,                                                                                     0,                                                                                     0],
[0, Piecewise(((2*theta - sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True)),            Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))],
[0,            Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)), Piecewise(((2*theta + sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (1, True))]])

In [9]:
y_depolarizing_component(psi,phi,alpha)


=== Analysis for Pauli Target: Y ===

--- Matrix L_k = U† Y U ---
Matrix([[-sin(s*theta), -I*cos(s*theta)], [I*cos(s*theta), sin(s*theta)]])

--- Pauli Basis Components f_k^(Y)(s) ---
f_Y^I(s) = 0
f_Y^X(s) = 0
f_Y^Y(s) = cos(s*theta)
f_Y^Z(s) = -sin(s*theta)

--- Variances and Covariances for L_k = Y ---
Var_I = 0
Var_X = 0
Var_Y = Piecewise(((theta + sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (1, True))
Var_Z = Piecewise(((theta - sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = 0
Cov_Y_Z = Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))


Matrix([
[0,                                                                                     0,                                                                                     0],
[0, Piecewise(((2*theta + sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (1, True)),           Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))],
[0,           Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)), Piecewise(((2*theta - sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True))]])

# Y-Gate

In [10]:
## Change for Each Gate implemented ##
psi = sp.pi / 2
phi = sp.pi /2

x_depolarizing_component(psi,phi,alpha)



=== Analysis for Pauli Target: X ===

--- Matrix L_k = U† X U ---
Matrix([[sin(s*theta), cos(s*theta)], [cos(s*theta), -sin(s*theta)]])

--- Pauli Basis Components f_k^(X)(s) ---
f_X^I(s) = 0
f_X^X(s) = cos(s*theta)
f_X^Y(s) = 0
f_X^Z(s) = sin(s*theta)

--- Variances and Covariances for L_k = X ---
Var_I = 0
Var_X = Piecewise(((theta + sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (1, True))
Var_Y = 0
Var_Z = Piecewise(((theta - sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Cov_Y_Z = 0


Matrix([
[Piecewise(((2*theta + sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (1, True)), 0,            Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))],
[                                                                                    0, 0,                                                                                     0],
[           Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)), 0, Piecewise(((2*theta - sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True))]])

In [11]:
y_depolarizing_component(psi,phi,alpha)


=== Analysis for Pauli Target: Y ===

--- Matrix L_k = U† Y U ---
Matrix([[0, -I], [I, 0]])

--- Pauli Basis Components f_k^(Y)(s) ---
f_Y^I(s) = 0
f_Y^X(s) = 0
f_Y^Y(s) = 1
f_Y^Z(s) = 0

--- Variances and Covariances for L_k = Y ---
Var_I = 0
Var_X = 0
Var_Y = 1
Var_Z = 0
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = 0
Cov_Y_Z = 0


Matrix([
[0, 0, 0],
[0, 1, 0],
[0, 0, 0]])

In [12]:
z_depolarizing_component(psi,phi,alpha)


=== Analysis for Pauli Target: Z ===

--- Matrix L_k = U† Z U ---
Matrix([[cos(s*theta), -sin(s*theta)], [-sin(s*theta), -cos(s*theta)]])

--- Pauli Basis Components f_k^(Z)(s) ---
f_Z^I(s) = 0
f_Z^X(s) = -sin(s*theta)
f_Z^Y(s) = 0
f_Z^Z(s) = cos(s*theta)

--- Variances and Covariances for L_k = Z ---
Var_I = 0
Var_X = Piecewise(((theta - sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Var_Y = 0
Var_Z = Piecewise(((theta + sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (1, True))
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Cov_Y_Z = 0


Matrix([
[Piecewise(((2*theta - sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True)), 0,           Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))],
[                                                                                    0, 0,                                                                                     0],
[          Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)), 0, Piecewise(((2*theta + sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (1, True))]])

# Z-Gate

In [13]:
## Change for Each Gate implemented ##
psi = 0
phi = sp.pi /2

x_depolarizing_component(psi,phi,alpha)



=== Analysis for Pauli Target: X ===

--- Matrix L_k = U† X U ---
Matrix([[0, exp(I*s*theta)], [exp(-I*s*theta), 0]])

--- Pauli Basis Components f_k^(X)(s) ---
f_X^I(s) = 0
f_X^X(s) = cos(s*theta)
f_X^Y(s) = -sin(s*theta)
f_X^Z(s) = 0

--- Variances and Covariances for L_k = X ---
Var_I = 0
Var_X = Piecewise(((theta + sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (1, True))
Var_Y = Piecewise(((theta - sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Var_Z = 0
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Cov_X_Z = 0
Cov_Y_Z = 0


Matrix([
[Piecewise(((2*theta + sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (1, True)),           Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)), 0],
[          Piecewise((-sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)), Piecewise(((2*theta - sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True)), 0],
[                                                                                    0,                                                                                     0, 0]])

In [14]:
y_depolarizing_component(psi,phi,alpha)


=== Analysis for Pauli Target: Y ===

--- Matrix L_k = U† Y U ---
Matrix([[0, -I*exp(I*s*theta)], [I*exp(-I*s*theta), 0]])

--- Pauli Basis Components f_k^(Y)(s) ---
f_Y^I(s) = 0
f_Y^X(s) = sin(s*theta)
f_Y^Y(s) = cos(s*theta)
f_Y^Z(s) = 0

--- Variances and Covariances for L_k = Y ---
Var_I = 0
Var_X = Piecewise(((theta - sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Var_Y = Piecewise(((theta + sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (1, True))
Var_Z = 0
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Cov_X_Z = 0
Cov_Y_Z = 0


Matrix([
[Piecewise(((2*theta - sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True)),            Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)), 0],
[           Piecewise((sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)), Piecewise(((2*theta + sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (1, True)), 0],
[                                                                                    0,                                                                                     0, 0]])

In [15]:
z_depolarizing_component(psi,phi,alpha)


=== Analysis for Pauli Target: Z ===

--- Matrix L_k = U† Z U ---
Matrix([[1, 0], [0, -1]])

--- Pauli Basis Components f_k^(Z)(s) ---
f_Z^I(s) = 0
f_Z^X(s) = 0
f_Z^Y(s) = 0
f_Z^Z(s) = 1

--- Variances and Covariances for L_k = Z ---
Var_I = 0
Var_X = 0
Var_Y = 0
Var_Z = 1
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = 0
Cov_Y_Z = 0


Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 1]])

# I Gate

In [19]:
theta = sp.pi *2
alpha = theta *s
x_depolarizing_component(psi,phi,alpha)



=== Analysis for Pauli Target: X ===

--- Matrix L_k = U† X U ---
Matrix([[0, exp(4*I*pi*s)], [exp(-4*I*pi*s), 0]])

--- Pauli Basis Components f_k^(X)(s) ---
f_X^I(s) = 0
f_X^X(s) = cos(4*pi*s)
f_X^Y(s) = -sin(4*pi*s)
f_X^Z(s) = 0

--- Variances and Covariances for L_k = X ---
Var_I = 0
Var_X = 1/2
Var_Y = 1/2
Var_Z = 0
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = 0
Cov_Y_Z = 0


Matrix([
[1/2,   0, 0],
[  0, 1/2, 0],
[  0,   0, 0]])

In [20]:
y_depolarizing_component(psi,phi,alpha)


=== Analysis for Pauli Target: Y ===

--- Matrix L_k = U† Y U ---
Matrix([[0, -I*exp(4*I*pi*s)], [I*exp(-4*I*pi*s), 0]])

--- Pauli Basis Components f_k^(Y)(s) ---
f_Y^I(s) = 0
f_Y^X(s) = sin(4*pi*s)
f_Y^Y(s) = cos(4*pi*s)
f_Y^Z(s) = 0

--- Variances and Covariances for L_k = Y ---
Var_I = 0
Var_X = 1/2
Var_Y = 1/2
Var_Z = 0
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = 0
Cov_Y_Z = 0


Matrix([
[1/2,   0, 0],
[  0, 1/2, 0],
[  0,   0, 0]])

In [21]:
z_depolarizing_component(psi,phi,alpha)


=== Analysis for Pauli Target: Z ===

--- Matrix L_k = U† Z U ---
Matrix([[1, 0], [0, -1]])

--- Pauli Basis Components f_k^(Z)(s) ---
f_Z^I(s) = 0
f_Z^X(s) = 0
f_Z^Y(s) = 0
f_Z^Z(s) = 1

--- Variances and Covariances for L_k = Z ---
Var_I = 0
Var_X = 0
Var_Y = 0
Var_Z = 1
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = 0
Cov_X_Z = 0
Cov_Y_Z = 0


Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 1]])